# Integrazione dei dati Netflix con i dati IMDb

I dati si riferiscono agli show di Netflix tra il 1 dicembre 2020 e il 31 marzo 2021.

In [2]:
import pandas as pd
import re

Leggo i dataframe

In [3]:
# Dati da flixpatrol in csv

netflix = pd.read_csv("Data_flixpatrol/Dati_netflix_dicembre-aprile/netflix_dic-mar2021.csv", index_col = 0)
top_ten = pd.read_csv("Data_flixpatrol/Dati_netflix_dicembre-aprile/top_ten_dic-mar2021.csv", index_col = 0)
top_film = pd.read_csv("Data_flixpatrol/Dati_netflix_dicembre-aprile/top_ten_film_dic-mar2021.csv", index_col = 0)
top_serie = pd.read_csv("Data_flixpatrol/Dati_netflix_dicembre-aprile/top_ten_serie_dic-mar2021.csv", index_col = 0)

In [4]:
# Dati da IMDb

basics = pd.read_csv("Data_imdb/title_basics.csv", sep = "\t", low_memory = False)
ratings = pd.read_csv("Data_imdb/ratings.csv", sep = "\t")

In [5]:
basics.head(3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"


In [6]:
ratings.head(3)

,tconst,averageRating,numVotes
0,tt0000001,5.6,1693
1,tt0000002,6.0,210
2,tt0000003,6.5,1439


### Estraggo,correggo i titoli e li ordino in ordine alfabetico

In [7]:
titoli_raw = list(set(netflix['name']))  # titoli così come sono nel database

titoli = titoli_raw.copy()

__In tutto ho 208 titoli.__
Alcuni titoli nel database originario (contenuti in _titoli_raw_) sono scritti male, quindi li correggo. _titoli_raw_ conterrà i titoli originari, mentre _titoli_ i titoli corretti.

In [8]:
# Stampo i titoli scritti male (= con un apostrofo)

for t in titoli_raw:
    occ = re.compile('^.*&\#39;.*$')
    
    if occ.match(t):
        print(t)

Ma Rainey&#39;s Black Bottom
Dawson&#39;s Creek
I&#39;m Not a Killer
Finding &#39;Ohana
So My Grandma&#39;s a Lesbian!
The Queen&#39;s Gambit
Don&#39;t Listen
To All the Boys I&#39;ve Loved Before
A Dog&#39;s Way Home


In [9]:
# Correggo (in titoli) i titoli scritti male

for t in titoli:
    
    occ = re.compile('(^.*)(\&\#39;)(.*$)')
    found = occ.match(t)
    
    if found:
        titoli.remove(t)
        pulita = re.sub(r'(\&\#39;)',"\'" ,t)
        titoli.append(pulita)

In [10]:
# Ordino in ordine alfabetico

titoli = sorted(titoli)
titoli_raw = sorted(titoli_raw)

### Controllo i valori nulli
Nel database netflix non ho valori nulli nella colonna imdb

In [11]:
netflix[netflix['imdb'].isnull()]

,result,id,name,url,imdb,tmdb,premiere,type_id,type,country_id,...,streaming,from,to,ranking,ranking_last,value,value_last,value_total,countries,days


Neanche in ratings ho valori nulli nelle colonne averageRating e numVotes (quelle che ci interessano)

In [12]:
ratings[ratings['averageRating'].isnull()]

,tconst,averageRating,numVotes


In [13]:
ratings[ratings['numVotes'].isnull()]

,tconst,averageRating,numVotes


__Osservazione:__ Per 32 titoli non ho rating (bisogna metterli a mano).

I titoli senza rating corrispondente nel database imdb vengono aggiunti alla lista _missing_

In [14]:
limit = len(titoli)

In [31]:
missing = []

for i in range(limit):
    cod = 'tt' + str(netflix[netflix['name'] == titoli_raw[i]]['imdb'].iloc[0])
    
    df = ratings[ratings['tconst'] == cod]
    
    if len(df) == 0:
        missing .append(titoli[i])
    
# print(len(missing))

In [32]:
missing

['Blow',
 'Body of Lies',
 'Charlie and the Chocolate Factory',
 'Constantine',
 'Corpse Bride',
 "Dawson's Creek",
 'Dinner for Schmucks',
 'Dracula Untold',
 'Dumb and Dumber',
 'Failure to Launch',
 'Ghostbusters',
 'How the Grinch Stole Christmas',
 'Il Traditore',
 'Jurassic World',
 'Million Dollar Baby',
 'Mr. Peabody & Sherman',
 'Pet Sematary',
 'Pride & Prejudice',
 'Scarface',
 'Searching for Bobby Fischer',
 'Secret Magic Control Agency',
 'Spider-Man',
 'Spider-Man 2',
 'Spider-Man 3',
 'Sucker Punch',
 'The 40 Year Old Virgin',
 'The Big Bang Theory',
 'The Goonies',
 'The Pianist',
 'The Texas Chainsaw Massacre',
 'V for Vendetta',
 'ariana grande: excuse me, i love you']

## Creazione del nuovo dataframe¶
Usando la colonna imdb del database netflix, per ogni titolo estraggo l'id tconst.

In [43]:
# Dizionario che serve per creare il dataframe che ci interessa: titolo, anno, voto medio e numero dei voti

Diz = {'Title': [], 'Average_rating' : [], 'Num_votes' : [], 'Type' : []}

Aggiungo una colonna a __ratings_imdb__ : ogni riga vale 0 se è un film, 1 se è una serie.

In [44]:
limit

208

In [45]:
for i in range(limit): # limit = 208, il numero dei titoli
    
    t = titoli[i]
    
    Diz['Title'].append(t)
    tipo = netflix[netflix['name'] == titoli_raw[i]]['type'].iloc[0]
    
    # Se è un film aggiungo 0
    if tipo == "Movie":
        Diz['Type'].append(0)
        
    # Se è una serie tv aggiungo 1
    else:
        Diz['Type'].append(1)
        
    if t not in missing:
        
        # Codice imdb estratto dal db netflix (a cui aggiungere 'tt')
        codice_imdb = 'tt' + str(netflix[netflix['name'] == titoli_raw[i]]['imdb'].iloc[0])
        
        # Risultato della ricerca nel db ratings del codice del titolo
        df = ratings[ratings['tconst'] == codice_imdb]
        
        Diz['Average_rating'].append(df['averageRating'].iloc[0])
        Diz['Num_votes'].append(df['numVotes'].iloc[0])
    
    # Per i titoli che non si trovano nel db
    else:
        Diz['Average_rating'].append('NaN')
        Diz['Num_votes'].append('NaN')

Creo il dataframe __ratings_imdb__

In [46]:
ratings_imdb = pd.DataFrame(data = Diz)

In [47]:
ratings_imdb

,Title,Average_rating,Num_votes,Type
0,365 Days,3.2,54757,0
1,5 Is the Perfect Number,5.3,1075,0
2,A California Christmas,5.7,6543,0
3,A Dog's Way Home,6.7,15319,0
4,A Star is Born,7.6,340182,0
...,...,...,...,...
203,Yes Man,6.8,337907,0
204,You vs. Wild,6.7,2204,1
205,Zero Chill,5.6,630,1
206,"ariana grande: excuse me, i love you",NaN,NaN,0


## Titoli senza rating

Aggiungo a mano il rating e il numero dei votanti

In [50]:
# Così modifico una cella del database
# ratings_imdb.iloc[24]['Average_rating'] = 7.6

In [51]:
ratings_imdb.iloc[10:30]

,Title,Average_rating,Num_votes,Type
10,Alien Worlds,6.5,2683,1
11,All My Friends Are Dead,5.8,4265,0
12,Animals on the Loose: A You vs. Wild Interacti...,5.9,626,0
13,"As Above, So Below",6.2,79524,0
14,Asphalt Burning,5.1,1710,0
15,Ava,5.4,42264,0
16,Bad Trip,6.6,6715,0
17,Behind Her Eyes,7.2,32779,1
18,Belleville Cop,4.8,1906,0
19,Below Zero,6.2,13567,0


### Esporto il dataframe in un csv

In [53]:
ratings_imdb.to_csv("Data_flixpatrol/Dati_netflix_dicembre-aprile/ratings_imdb_dic-mar2021.csv")